In [2]:
def sim_option_price_sdf(series, S_t, K, horizon, par, draws, r, model):
    if model == "GARCH":
        omega = par[0]
        alpha = par[1]
        beta = par[2]
    S_T = np.zeros(draws)
    temp = np.zeros(draws)
        
    sigma2 = np.zeros(len(series))
    sigma2[0] = np.var(series)
        
    for i in range(len(series)):
        sigma2[i] = omega + alpha*series[i-1]**2 + beta*sigma2[0][i-1]
        
    for i in range(len(draws)):
        ret = np.zeros(horizon)
        theta = np.zeros(horizon)
        sdf = np.zeros(horizon)
            
        s = sigma2[-1]
        s =  omega + alpha*series[-1]**2 + beta*s
            
        for j in range(len(horizon)):
            theta[j] = r/s - 1/2
            ret[j] = (s)**0.5 * np.random.normal(0,1,1)[0]
            sdf[j] = np.exp(theta[j]*ret[j])/np.exp( (1+theta[j])**2 * s/2)
            s = omega + alpha*ret[j]**2 + beta*s
        
        rsum = np.sum(ret)
        S_T[i] = S_t * np.exp(rsum)
        prodsdf = np.product(sdf)
        temp[i] = prodsdf * (S_T[i]-K)*(S_T[i]>K)
    return np.mean(temp)